In [18]:
{3:{}}

{3: {}}

In [1]:
pdbs = ['7EOR',  '7EOS',  '7EOU']
path = '../0_initial_structure/'

In [2]:
three2one = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M'}

one2three = {v: k for k, v in three2one.items()}

def missing_residues(pdb):
    result = {'A': {},'B':{},'C':{}, 'D':{}}
    read = False
    with open(pdb, 'r') as file:
        for line in file:
            if line[:27] == 'REMARK 465   M RES C SSSEQI':
                read = True
            elif line[:10] == 'REMARK 465' and read:
                res, chain, res_id = line.split()[-3:]
                
                result[chain][int(res_id)] = three2one[res]
                
    return result
                
def remodel_file(file_out,pdb_chainX, missing_residues4chainX):
    
    ############################################################
    residues4chainX = {}
    res_id_old = -1
    with open(pdb_chainX, 'r') as file:
        for line in file:
            
            if line[:4] == 'ATOM':

                
                atom = line[13:16].split()[0]
                res = line[16:20].split()[0]
                res_id = line[22:26].split()[0]
                
                res_id = int(res_id)
                
                if atom == 'CA':
                    try:
                        r = three2one[res]
                        residues4chainX[res_id] = r
                    except KeyError:
                        print('errori',res)
    
                        if len(res) == 4:
                            if res[0] == 'A':
                                r = three2one[res[1:]]
                                print('added',res)
                                residues4chainX[res_id] = r
                        else:
                            raise ValueError
                                            
    min_res, max_res =  min(residues4chainX.keys()), max(residues4chainX.keys())
    
    
    ##################################################################
    residues_to_relax = set(missing_residues4chainX.keys())
    for res_id in missing_residues4chainX.keys():
        if min_res <= res_id <= max_res:
            for i in [-2,-1,1,2]:
                residues_to_relax.add(res_id+i)

    #################################################################
    seq = ''
    with open(file_out, 'w') as file:
        
        i=1
        for res_id in range( min_res, max_res +1 ):
            
            if res_id in residues4chainX:
                res = residues4chainX[res_id]
                seq += res
                if res_id in residues_to_relax:
                    line = '{} {} L PIKAA {}\n'.format(i,res,res)
                    file.write(line)
                
                elif res_id < max_res:
                    line = '{} {} .\n'.format(i,res)
                    file.write(line)
                elif res_id == max_res:
                    line = '{} {} .'.format(i,res)
                    file.write(line)
                else:
                    raise ValueError
                i += 1
                    
            elif res_id in missing_residues4chainX:
                res = missing_residues4chainX[res_id]
                seq += res
                line = '0 X L PIKAA {}\n'.format(res)
                file.write(line)
                
            else:
                raise ValueError
    return seq
                
                    


In [3]:
for pdb in pdbs:
    
    pdb_file = '{}/{}/{}.pdb'.format(path,pdb,pdb.lower())
    print(pdb_file)
    mr = missing_residues(pdb_file)
    
    for chain in 'ABCD':

        pdb_chainX = '{}/{}/{}_chain{}.pdb'.format(path,pdb,pdb.lower(),chain)
        print()
        print(pdb_chainX)
        file_out='{}/{}_chain{}.remodel'.format(pdb,pdb.lower(),chain)


        seq = remodel_file(file_out,pdb_chainX, mr[chain])
        print()
        print(seq)

../0_initial_structure//7EOR/7eor.pdb

../0_initial_structure//7EOR/7eor_chainA.pdb

LNIAVMLGHSHDVTERELRTLWGPEQAAGLPLDVNVVALLMNRTDPKSLITHVCDLMSGARIHGLVFGDDTDQEAVAQMLDFISSHTFVPILGIHGGASMIMADKDPTSTFFQFGASIQQQATVMLKIMQDYDWHVFSLVTTIFPGYREFISFVKTTVDNSFVGWDMQNVITLDTSFEDAKTQVQLKKIHSSVILLYCSKDEAVLILSEARSLGLTGYDFFWIVPSLVSGNTELIPKEFPSGLISVSYDDWDYSLEARVRDGIGILTTAASSMLEKFSYIPEAKASCYGQMERPEVPMHTLHPFMVNVTWDGKDLSFTEEGYQVHPRLVVIVLNKDREWEKVGKWENHTLSLRHAVWPRYKSFSDCEPDDNHLSIVTLEEAPFVIVEDIDPLTETCVRNTVPCRKFVKINNSTNEGMNVKKCCKGFCIDILKKLSRTVKFTYDLYLVTNGKHGKKVNNVWNGMIGEVVYQRAVMAVGSLTINEERSEVVDFSVPFVETGISVMVSRSNGTVSPSAFLEPFSASVWVMMFVMLLIVSAIAVFVFEYFSPVGYNRNLAKGKAPHGPSFTIGKAIWLLWGLVFNNSVPVQNPKGTTSKIMVSVWAFFAVIFLASYTANLAAFMIQEEFVDQVTGLSDKKFQRPHDYSPPFRFGTVPNGSTERNIRNNYPYMHQYMTKFNQKGVEDALVSLKTGKLDAFIYDAAVLNYKAGRDEGCKLVTIGSGYIFATTGYGIALQKGSPWKRQIDLALLQFVGDGEMEELETCWLTGICHNEKNEVMSSQLDIDNMAGVFYMLAAAMALSLITFIW

../0_initial_structure//7EOR/7eor_chainB.pdb
errori AILE
added AILE
errori BILE

KIVNIGAVLSTRKHEQMFREAVNQANKR

In [4]:
nstruct = 1000
for pdb in pdbs:
    for chain in 'ABCD':
        file_out='{}/flag_missing_loops_{}'.format(pdb,chain)
        
        with open(file_out, 'w') as file:
            lines = '-in:file:s ../../0_initial_structure/{}/{}_chain{}.pdb\n'.format(pdb,pdb.lower(),chain)
            lines += '-remodel:blueprint {}_chain{}.remodel\n\n'.format(pdb.lower(),chain)
            lines += '-nstruct {}\n\n'.format(nstruct)
            lines += '-run:chain A\n'
            lines += '-remodel:num_trajectory 1\n'
            lines += '-remodel:quick_and_dirty\n\n'
            lines += '-out:path:all output_files\n'
            lines += '-out:file:scorefile {}_chain{}.sc'.format(pdb.lower(),chain)
            file.write(lines)